In [52]:
import networkx as nx
import math

In [8]:
# a)  Read the file “old_edges.txt” and create a graph using NetworkX.
fh = open("old_edges.txt", 'r')
g_old = nx.read_edgelist(fh, delimiter = '\t')
print(nx.info(g_old))

Name: 
Type: Graph
Number of nodes: 6427
Number of edges: 15791
Average degree:   4.9140


In [19]:
# b) Read the file "new_edges.txt"
fh2 = open("new_edges.txt", 'r')
g_new = nx.read_edgelist(fh2, delimiter = '\t')
print(nx.info(g_new))

# keep track of the new collaborations
g_new.degree()

Name: 
Type: Graph
Number of nodes: 1143
Number of edges: 1701
Average degree:   2.9764


DegreeView({'Aaditya Ramdas': 3, 'Kevin G. Jamieson': 2, 'Martin J. Wainwright': 4, 'Michael I. Jordan': 16, 'Aapo Hyvärinen': 1, 'Motoaki Kawanabe': 2, 'Aaron C. Courville': 11, 'Simon Lacoste-Julien': 5, 'Aaron Roth': 3, 'Bo Waggoner': 3, 'Aarti Singh': 10, 'Artur Dubrawski': 3, 'Barnabás Póczos': 9, 'Eric P. Xing': 7, 'Abdel-rahman Mohamed': 1, 'Pushmeet Kohli': 19, 'Adam Lerer': 2, 'Arthur Szlam': 3, 'Adam R. Klivans': 2, 'Raghu Meka': 1, 'Adam Santoro': 5, 'David G. T. Barrett': 6, 'Mateusz Malinowski': 5, 'Peter Battaglia': 18, 'Razvan Pascanu': 28, 'Tim Lillicrap': 10, 'Adarsh Prasad': 1, 'Alexandru Niculescu-Mizil': 2, 'Adith Swaminathan': 4, 'Akshay Krishnamurthy': 4, 'Alekh Agarwal': 6, 'John Langford 0001': 6, 'Miroslav Dudík': 12, 'Aditi Raghunathan': 3, 'Gregory Valiant': 4, 'James Zou': 6, 'Prateek Jain 0002': 5, 'Aditya Grover': 2, 'Jayesh K. Gupta': 2, 'Maruan Al-Shedivat': 4, 'Adrian Weller': 14, 'Jinwoo Shin': 2, 'Adrià Puigdomènech Badia': 16, 'Charles Blundell': 7, 

In [57]:
# c) Recommend by number of common friends
def friends(G, X):
    """Returns a set of the friends of the given author, in the given graph"""
    return set(G.neighbors(X))

def friends_of_friends(G, X):
    """Returns a set of friends of friends of the given author, in the given graph"""
    X_friends = friends(G, X)
    X_friends_of_friends = set()

    for friend in X_friends:
        X_friends_of_friends.update(friends(G, friend) - X_friends)

    return X_friends_of_friends - {X}

def common_friends(G, X1, X2):
    """Returns the set of friends that X1 and X2 have in common."""
    return friends(G, X1).intersection(friends(G, X2))

def common_friends_number(G, X):
    common_friends_dict = {}
    X_friends_of_friends = friends_of_friends(G, X)

    for y in X_friends_of_friends:
        number = len(common_friends(G, X, y))
        if number >= 1:
            common_friends_dict[y] = number

    sorted_list = [v[0] for v in sorted(common_friends_dict.items(), key=lambda kv: kv[1], reverse=True)]
    
    return sorted_list

In [50]:
# d) Make recommendations using Jaccard’s Index
def jaccard_index(G, X):
    jaccard_index_dict = {}
    X_friends = friends(G, X)
    X_friends_of_friends = friends_of_friends(G, X)
    
    for y in X_friends_of_friends:
        y_friends = friends(G, y)
        intersection = len(common_friends(G, X, y))
        union = (len(X_friends) + len(y_friends)) - intersection
        jaccard_score = intersection / union
        jaccard_index_dict[y] = jaccard_score

    sorted_list = [v[0] for v in sorted(jaccard_index_dict.items(), key=lambda kv: kv[1], reverse=True)]
        
    return sorted_list

In [53]:
# e) Make recommendations using Adamic/Adar Index
def adamic_adar_index(G, X):
    X_friends = friends(G, X)
    X_friends_of_friends = friends_of_friends(G, X)
    adamic_adar_index_dict = {}
    
    for y in X_friends_of_friends:
        common_neighbors_Xy = common_friends(G, X, y)
        
        adamic_adar_index = 0
        for Z in common_neighbors_Xy:
            Z_friends = friends(G, Z)
            adamic_adar_index += 1 / math.log(len(Z_friends))
            
        adamic_adar_index_dict[y] = adamic_adar_index
        
    sorted_list = [v[0] for v in sorted(adamic_adar_index_dict.items(), key=lambda kv: kv[1], reverse=True)]
    
    return sorted_list

In [85]:
# f) How good are the recommendations we make?

# pick those authors that formed at least 10 new connections between 2017-2018. 
pick = [node for node,degree in dict(g_new.degree()).items() if degree >= 10]

# Approach 1 - for making recommendations using number of common friends
accuracy_dict = {}

for X in pick:
    # take the 10 first recommendations for each user
    recomm = common_friends_number(g_old, X)[0:10]
    X_true_friends = friends(g_new, X)
    
    # calculate the number of them that were actually formed during 2017-2018
    found_in_true_friends = set(recomm).intersection(X_true_friends)
    accuracy_dict[X] = len(found_in_true_friends) / len(X_true_friends)
    
# calculate the average among users
score = [v for _, v in accuracy_dict.items()]
average = sum(score) / len(score)
print(average)

0.07404924014963284


In [86]:
# Approach 1 - for making recommendations using Jaccard Index
accuracy_dict = {}

for X in pick:
    # take the 10 first recommendations for each user
    recomm = jaccard_index(g_old, X)[0:10]
    X_true_friends = friends(g_new, X)
    
    # calculate the number of them that were actually formed during 2017-2018    
    found_in_true_friends = set(recomm).intersection(X_true_friends)
    accuracy_dict[X] = len(found_in_true_friends) / len(X_true_friends)
    
# calculate the average among users
score = [v for _, v in accuracy_dict.items()]
average = sum(score) / len(score)
print(average)

0.04847337437328376


In [87]:
# Approach 1 - for making recommendations using Adamic/Adar Index
accuracy_dict = {}

for X in pick:
    # take the 10 first recommendations for each user
    recomm = adamic_adar_index(g_old, X)[0:10]
    X_true_friends = friends(g_new, X)
    
    # calculate the number of them that were actually formed during 2017-2018    
    found_in_true_friends = set(recomm).intersection(X_true_friends)
    accuracy_dict[X] = len(found_in_true_friends) / len(X_true_friends)
    
# calculate the average among users
score = [v for _, v in accuracy_dict.items()]
average = sum(score) / len(score)
print(average)

0.06629307606654132


In [91]:
# Approach 2 - for making recommendations using number of common friends
rank_dict = {}

for X in pick:
    recomm = common_friends_number(g_old, X)
    X_true_friends = friends(g_new, X)
    
    # for each newly formed collaboration of user X, calculate the rank of this collaboration  
    for new in X_true_friends:
        if new in recomm:
            rank = recomm.index(new)
        else:
            rank = len(recomm)
    
        rank_dict[new] = rank
    
# calculate the average among newly formed edges
values = [v for _, v in rank_dict.items()]
average = sum(values) / len(values)
print(average)

101.1275720164609


In [93]:
# Approach 2 - for making recommendations using Jaccard Index
rank_dict = {}

for X in pick:
    recomm = jaccard_index(g_old, X)
    X_true_friends = friends(g_new, X)
    
    # for each newly formed collaboration of user X, calculate the rank of this collaboration  
    for new in X_true_friends:
        if new in recomm:
            rank = recomm.index(new)
        else:
            rank = len(recomm)
    
        rank_dict[new] = rank
    
# calculate the average among newly formed edges
values = [v for _, v in rank_dict.items()]
average = sum(values) / len(values)
print(average)

102.60082304526749


In [94]:
# Approach 2 - for making recommendations using Adamic/Adar Index
rank_dict = {}

for X in pick:
    recomm = adamic_adar_index(g_old, X)
    X_true_friends = friends(g_new, X)
    
    # for each newly formed collaboration of user X, calculate the rank of this collaboration  
    for new in X_true_friends:
        if new in recomm:
            rank = recomm.index(new)
        else:
            rank = len(recomm)
    
        rank_dict[new] = rank
    
# calculate the average among newly formed edges
values = [v for _, v in rank_dict.items()]
average = sum(values) / len(values)
print(average)

99.66666666666667
